In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")


In [86]:
# transform_train = transforms.Compose([transforms.ToTensor()])
transform_train = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.137,), (0.3081,))])
transform_valid = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transform_train)#, target_transform=lambda x: torch.Tensor([x]).float())
trainloader = DataLoader(trainset, batch_size=4,
                         shuffle=True, num_workers=0)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transform_valid) #target_transform=lambda x: torch.Tensor([x]).float())
testloader = DataLoader(testset, batch_size=20,
                        shuffle=False, num_workers=0)


In [87]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=0)
        self.flatten = nn.Flatten(1)
        self.fc1 = nn.Linear(12 * 12 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        # x = self.pool(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = F.dropout(x, 0.25)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

metric_func = lambda y_pred, y_true: roc_auc_score(y_true=y_true.data.numpy(), y_score=y_pred.data.numpy())
matric_name = "auc"
def train_step(model: nn.Module, features:torch.Tensor, labels: torch.Tensor):
    features = features.to(device=device)
    labels = labels.to(device)
    
    # model.train()
    optimizer.zero_grad()
    y_pred = model(features)
    loss = criterion(y_pred, labels)
    # metric = metric_func(y_pred, labels)
    metric = torch.Tensor(0)
    loss.backward()
    optimizer.step()
    return loss, 0


In [88]:
from sklearn.metrics import roc_auc_score

cnn = Cnn()
cnn.to(device)
#criterion = nn.CrossEntropyLoss()
criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(cnn.parameters(), lr=0.1)

In [89]:
import torchkeras
input_shape = (1, 28, 28)
torchkeras.summary(Cnn(), input_shape=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Conv2d-2           [-1, 64, 24, 24]          18,496
         MaxPool2d-3           [-1, 64, 12, 12]               0
           Flatten-4                 [-1, 9216]               0
            Linear-5                  [-1, 128]       1,179,776
            Linear-6                   [-1, 10]           1,290
Total params: 1,199,882
Trainable params: 1,199,882
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.002991
Forward/backward pass size (MB): 0.587967
Params size (MB): 4.577187
Estimated Total Size (MB): 5.168144
----------------------------------------------------------------


In [90]:
(features, labels) = next(iter(trainloader))
# print((features[0], labels[0]))
print(trainloader.batch_sampler)
print(len(trainloader))

15000


In [91]:
for epoch in range(20):
    loss_sum = []
    metric_sum = 0.0
    total_loss = 0.
    for i, data in enumerate(trainloader):
        (features, labels) = data
        loss, metric = train_step(cnn, features=features, labels=labels)
        loss_sum.append(loss)
        metric_sum += metric
        total_loss += loss.item()
        loss_sum.append(loss.item())
        loss.retain_grad()
        if i % 10 == 0:
            print('[%d, %5d] loss: %.5f' %
                    (epoch + 1, i + 1, torch.mean(torch.Tensor(loss_sum))))
            total_loss = 0.0


[1,     1] loss: 2.38894
[1,    11] loss: 2.40372
[1,    21] loss: 2.31416
[1,    31] loss: 2.28675
[1,    41] loss: 2.21481
[1,    51] loss: 2.15304
[1,    61] loss: 2.06059
[1,    71] loss: 1.97068
[1,    81] loss: 1.96126
[1,    91] loss: 1.87647
[1,   101] loss: 1.83211
[1,   111] loss: 1.79312
[1,   121] loss: 1.75804
[1,   131] loss: 1.71103
[1,   141] loss: 1.69429
[1,   151] loss: 1.69634
[1,   161] loss: 1.68789
[1,   171] loss: 1.64760
[1,   181] loss: 1.62591
[1,   191] loss: 1.58942
[1,   201] loss: 1.56226
[1,   211] loss: 1.52155
[1,   221] loss: 1.51625
[1,   231] loss: 1.47697
[1,   241] loss: 1.45734
[1,   251] loss: 1.44413
[1,   261] loss: 1.43355
[1,   271] loss: 1.41455
[1,   281] loss: 1.39888
[1,   291] loss: 1.37891
[1,   301] loss: 1.36983
[1,   311] loss: 1.35324
[1,   321] loss: 1.33943
[1,   331] loss: 1.33362
[1,   341] loss: 1.32788
[1,   351] loss: 1.31338
[1,   361] loss: 1.29428
[1,   371] loss: 1.27661
[1,   381] loss: 1.26332
[1,   391] loss: 1.26177
